In [55]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv('Expanded_data_with_more_features.csv')
df.head()

,Unnamed: 0,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,1,female,group C,some college,standard,NaN,married,sometimes,yes,0.0,NaN,5 - 10,69,90,88
2,2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,NaN,5 - 10,45,56,42
4,4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75


In [40]:
# 1 Скільки студентів і студенток у наборі даних?
fm = df.query('Gender == "female"')['Gender'].count()
ml = df.query('Gender == "male"')['Gender'].count()
print(f"Студентів = {ml}, Студенток = {fm}")


Студентів = 15217, Студенток = 15424


In [41]:
# 2 Який відсоток студентів чоловічої статі в наборі даних?
male = df.query('Gender == "male"')['Gender'].count()
female = df.query('Gender == "female"')['Gender'].count()
male / (male + female) * 100

49.6622172905584

In [42]:
# 3 Який відсоток студенток у наборі даних?
male = df.query('Gender == "male"')['Gender'].count()
female = df.query('Gender == "female"')['Gender'].count()
female / (male + female) * 100

50.337782709441605

In [43]:
# 4 Яка етнічна група має найвищий середній бал з математики? 
df.groupby('EthnicGroup')["MathScore"].mean().sort_values(ascending=False)[:1]

EthnicGroup
group E    75.298936
Name: MathScore, dtype: float64

In [44]:
# 5 Яка етнічна група має найвищий середній бал читання?
df.groupby('EthnicGroup')["ReadingScore"].mean().sort_values(ascending=False)[:1]

EthnicGroup
group E    74.251423
Name: ReadingScore, dtype: float64

In [45]:
# 6 Яка етнічна група має найвищий середній бал з письма?
df.groupby('EthnicGroup')["WritingScore"].mean().sort_values(ascending=False)[:1]

EthnicGroup
group E    72.67706
Name: WritingScore, dtype: float64

In [46]:
# 7 Який середній бал з математики для студентів, які закінчили курс підготовки до тесту?
df.query("TestPrep == 'completed'")["MathScore"].mean()

69.5466599698644

In [47]:
# 8. Який середній бал з читання для студентів, які пройшли курс підготовки до тесту?
df.query("TestPrep == 'completed'")["ReadingScore"].mean()

73.73299849321948

In [48]:
# 9 Який середній бал для студентів, які закінчили курс підготовки до тесту?
df['MeanTestsScore'] = (df['MathScore'] + df['ReadingScore'] + df['WritingScore'])/3
df.query("TestPrep == 'completed'")["MeanTestsScore"].mean()

72.66097438473129

In [49]:
# 10 Скільки студентів мають одружених батьків?
df.query("ParentMaritalStatus == 'married'").count()['ParentMaritalStatus']

16844

In [50]:
# 11 Скільки студентів мають батьків-одинаків?
df.query("ParentMaritalStatus == 'single'").count()['ParentMaritalStatus']

7097

In [51]:
# 12 Чи студенти, які регулярно займаються спортом, мають вищі середні результати з математики, ніж ті, хто ніколи не займається спортом?
reg = df.query("PracticeSport == 'regularly'")["MathScore"].mean()
nev = df.query("PracticeSport == 'never'")["MathScore"].mean()
reg > nev

True

In [52]:
# 13 Чи студенти, які регулярно займаються спортом, мають вищі середні результати читання, ніж ті, хто ніколи не займається спортом?
reg = df.query("PracticeSport == 'regularly'")["ReadingScore"].mean()
nev = df.query("PracticeSport == 'never'")["ReadingScore"].mean()
reg > nev

True

In [53]:
# 14 Скільки учнів користується шкільним автобусом, щоб дістатися до школи?
df.query('TransportMeans == "school_bus"')['TransportMeans'].count() 

16145

In [54]:
# 15 Скільки учнів користується приватним транспортом, щоб дістатися до школи?
df.query('TransportMeans == "private"')['TransportMeans'].count()

11362

In [56]:
from sklearn.preprocessing import StandardScaler
import sklearn as sk
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

In [57]:
cols = [
    'Gender', 
    'EthnicGroup', 
    'ParentEduc', 
    'LunchType', 
    'ParentMaritalStatus', 
    'PracticeSport', 
    'IsFirstChild', 
    'NrSiblings',       
    'TransportMeans', 
    'WklyStudyHours', 
    'MathScore', 
    'ReadingScore', 
    'WritingScore'
]

In [58]:
df['IsTestCompleted'] = df['TestPrep'] == 'completed'

In [59]:
df[cols + ['IsTestCompleted']]

,Gender,EthnicGroup,ParentEduc,LunchType,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,IsTestCompleted
0,female,NaN,bachelor's degree,standard,married,regularly,yes,3.0,school_bus,< 5,71,71,74,False
1,female,group C,some college,standard,married,sometimes,yes,0.0,NaN,5 - 10,69,90,88,False
2,female,group B,master's degree,standard,single,sometimes,yes,4.0,school_bus,< 5,87,93,91,False
3,male,group A,associate's degree,free/reduced,married,never,no,1.0,NaN,5 - 10,45,56,42,False
4,male,group C,some college,standard,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30636,female,group D,high school,standard,single,sometimes,no,2.0,school_bus,5 - 10,59,61,65,False
30637,male,group E,high school,standard,single,regularly,no,1.0,private,5 - 10,58,53,51,False
30638,female,NaN,high school,free/reduced,married,sometimes,no,1.0,private,5 - 10,61,70,67,True
30639,female,group D,associate's degree,standard,married,regularly,no,3.0,school_bus,5 - 10,82,90,93,True


In [63]:
num_features = [
    'MathScore', 
    'ReadingScore', 
    'WritingScore',
    'NrSiblings'
]
cat_features = [
    'Gender', 
    'EthnicGroup', 
    'ParentEduc', 
    'LunchType', 
    'ParentMaritalStatus', 
    'PracticeSport',
    'IsFirstChild',
    'TransportMeans',
    'WklyStudyHours'
]

In [64]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="mean")),
    ('scaler', StandardScaler()),
])

pipeline = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", OneHotEncoder(), cat_features)
])



X = pipeline.fit_transform(df)
Y = df['IsTestCompleted'].values

In [65]:
model = LogisticRegression()
cv = ShuffleSplit(n_splits=15, test_size=0.25, random_state=0)
cross_val_score(LogisticRegression(), X, Y, cv=cv).mean()

0.7495888265239525